# Heston Basket Options  

A basket option is a type of financial derivative where the underlying asset is a basket (group), of commodities, securities, or currencies. In some sense, it is a list of assests associated with a single option. Thus, our dataset will have a list of assest prices and correlations along with the other parameters in the normal case.   

In [5]:
import os
import time
import pathlib
import numpy as np
import pandas as pd

from heston import *
from df_helper import *
from blackscholes import *

We first load the data to generate the baskets of options using `basket_data.csv`.  

Note that the data here will be inputs to our Heston model, which will give simulated option prices for the training of our neural network.  

In [6]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/basket_data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,mean_volatility,reversion,var_of_vol,rho
0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,2.0,1.163095,0.053646,0.022956,0.047665,1.741895,0.285332,-0.557252
1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,2.0,0.997845,0.053646,0.114784,0.018427,0.765240,0.304296,-0.355317
2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,2.0,0.997223,0.053646,0.106823,0.021562,1.578755,0.553407,-0.728337
3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,2.0,1.002110,0.053646,0.110336,0.013416,2.370529,0.622139,-0.421136
4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,2.0,0.992831,0.053646,0.110336,0.011473,3.822598,0.616489,-0.572024


## Test implementation

We generate basket options below:

In [7]:
dim  = 7     # Dimension of basket (number of stocks)
num_samples = 1000
inputs_array = generate_inputs_nn(df, dim, num_samples)
inputs = pd.DataFrame(inputs_array, columns = ['underlyings_price', 'rho', 'days_to_maturity', 
                        'strike', 'volatility','mean_volatility','reversion', 'vol_of_var','rate'])
print(inputs.shape)
inputs.head()

(1000, 9)


,underlyings_price,rho,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate
0,"[5.609299926757813, 4.368099975585937, 4.71720...","[-0.2568167516127495, -0.23752191383923893, -0...",52.0,4.5,0.023013,0.030609,4.954257,0.544072,0.002241
1,"[20.101500244140624, 15.8725, 16.3272497558593...","[-0.08101871895986272, -0.1380695985531425, -0...",77.0,5.6,0.005833,0.024173,0.540823,0.094257,0.052515
2,"[38.08570068359375, 15.52262939453125, 16.3605...","[-0.10745472257478458, -0.23401761775111257, -...",121.0,11.0,0.050875,0.038243,3.996658,0.693612,0.007357
3,"[4.387200012207031, 14.4225, 4.638200073242188...","[-0.10593952858004502, -0.4332986887223966, -0...",106.0,71.5,0.002178,0.016408,2.74288,0.328159,0.012324
4,"[418.42, 15.706500244140624, 13.83112060546875...","[-0.7393055687670563, -0.2063186248118251, -0....",147.0,16.75,0.020151,0.002684,2.537333,0.564929,0.007098


### Test the scalar case

In [8]:
S_0   = inputs['underlyings_price'].values[0] 
v_0   = inputs['volatility'].values[0] 
r     = inputs['rate'].values[0] 
theta = inputs['mean_volatility'].values[0] 
kappa = inputs['reversion'].values[0] 
xi    = inputs['vol_of_var'].values[0] 
K     = inputs['strike'].values[0] 
rho   = inputs['rho'].values[0] 
T     = inputs['days_to_maturity'].values[0] 

scalar_result = generate_heston_paths_test(S_0, T, K, r, kappa, theta, v_0, rho, xi, 
                          100, 1000)
print(f"The scalar result is {scalar_result}")

The scalar result is 6.501285395822575


### Test the vector case

In [9]:
inputs.columns.values[2:]

array(['days_to_maturity', 'strike', 'volatility', 'mean_volatility',
       'reversion', 'vol_of_var', 'rate'], dtype=object)

In [10]:
begin = time.time()
result = generate_heston_paths_vec(inputs, num_samples, steps=100, num_sims=1000)
end = time.time()
print(f"The vector result is {result[0]}")
print(f"The shape of the result is {result.shape}")
print(f"Time taken is {end-begin} seconds")

The vector result is 6.391481383863025
The shape of the result is (1000,)
Time taken is 39.34797811508179 seconds


We can see that the first element of `generate_heston_paths_vec` is approximately the same as `generate_heston_paths`. This verified that our models are correct.

## Processing  

We now make the dataframe into the desired form for training the neural network

In [11]:
inputs['contract_price'] = result
inputs = flattenDim(7, inputs)
inputs.head()

,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate,contract_price,Underlying_0,Underlying_1,...,Underlying_4,Underlying_5,Underlying_6,Rho_0,Rho_1,Rho_2,Rho_3,Rho_4,Rho_5,Rho_6
0,52.0,4.50,0.023013,0.030609,4.954257,0.544072,0.002241,6.391481,5.609300,4.368100,...,5.508500,14.125000,21.507500,-0.256817,-0.237522,-0.519130,-0.672251,-0.147610,-0.472888,-0.576727
1,77.0,5.60,0.005833,0.024173,0.540823,0.094257,0.052515,10.890161,20.101500,15.872500,...,5.097700,5.433400,4.833900,-0.081019,-0.138070,-0.496760,-0.608802,-0.737819,-0.317828,-0.268578
2,121.0,11.00,0.050875,0.038243,3.996658,0.693612,0.007357,33.197303,38.085701,15.522629,...,56.716001,11.949399,56.777002,-0.107455,-0.234018,-0.140495,-0.125982,-0.530276,-0.074410,-0.245753
3,106.0,71.50,0.002178,0.016408,2.742880,0.328159,0.012324,2.912927,4.387200,14.422500,...,5.460200,13.073130,6.700000,-0.105940,-0.433299,-0.684673,-0.409936,-0.720678,-0.226944,-0.711910
4,147.0,16.75,0.020151,0.002684,2.537333,0.564929,0.007098,56.492795,418.420000,15.706500,...,4.235600,5.549500,20.665750,-0.739306,-0.206319,-0.599167,-0.557574,-0.079290,-0.262978,-0.103366


In [12]:
# uncomment to save the data
#inputs.to_csv(os.path.join(parent_path, f'data/small_basket_data_{dim}.csv'))

Generate dataframe for each required dimension and store it:

In [23]:
dims = [1, 4, 7, 10, 13, 16]
generate_and_store(df, dims, 1000, store_time = True, store_data=True)

In [22]:
generate_and_store(df, dims, 50, store_time = True, store_data=True)